# Stochastic Multi-armed bandit algorithms

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

random.seed(0)

In [ ]:
def ind_max(temp):
    m = max(temp)
    list_m = list(filter(lambda x: temp[x] == m, range(len(temp))))
    return random.choice(list_m)

In [ ]:
def categorical_draw(probs):
    z = random.random()
    cum_prob = 0.0
    for i in range(len(probs)):
        prob = probs[i]
        cum_prob += prob
        if cum_prob > z:
            return i
  
    return len(probs) - 1

# Environment

In [ ]:
class BernoulliArm():
    def __init__(self,p):
        self.p = p
        
    def draw(self):
        if random.random() > self.p:
            return 0.0
        else:
            return 1.0

# Algorithms

## Random

In [ ]:
class Random():
    def __init__(self,counts,values):
        self.counts = counts # list that have the number of pulling each arm
        self.values = values # list for expected reward for each arm
        return
    
    def initialize(self,n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        return
    
    def select_arm(self):
        return random.randrange(len(self.values))

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]
    
        value = self.values[chosen_arm]
        new_value = ((n-1)/float(n))*value + (1/float(n)) * reward
        self.values[chosen_arm] = new_value 
        return


## Annealing epsilon greedy

In [ ]:
class AnnealingEpsilonGreedy():
    def __init__(self, counts, values):
        self.counts = counts
        self.values = values
        return
    
    def initialize(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        return
    
    def select_arm(self):
        t = sum(self.counts) + 1 
        epsilon = 1 / math.log(t + 0.0000001)
        
        if random.random() > epsilon:
            return ind_max(self.values)
        else:
            return random.randrange(len(self.values))
    
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]

        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value
        return

## Annealing softmax

In [ ]:
class AnnealingSoftmax:
    def __init__(self, counts, values):
        self.counts = counts
        self.values = values
        return
  
    def initialize(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        return
  
    def select_arm(self):
        t = sum(self.counts) + 1
        temperature = 1 / math.log(t + 0.0000001) 
        z = sum([math.exp(v / temperature) for v in self.values])
        probs = [math.exp(v / temperature) / z for v in self.values]
        return categorical_draw(probs)

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value
        return

## UCB1

In [ ]:
class UCB1():
    def __init__(self, counts, values): 
        self.counts = counts
        self.values = values
        return
  
    def initialize(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        return

    def select_arm(self):
        n_arms = len(self.counts)
        for arm in range(n_arms):
            if self.counts[arm] == 0: # pull all arms at least once
                return arm

        ucb_values = [0.0 for arm in range(n_arms)]
        total_counts = sum(self.counts)
        for arm in range(n_arms):
            bonus = math.sqrt((2 * math.log(total_counts)) / float(self.counts[arm]))
            ucb_values[arm] = self.values[arm] + bonus
        return ind_max(ucb_values)
  
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value
        return

## UCB-Tuned

In [ ]:
class UCBTuned():
    def __init__(self, counts, values, reward_squared): 
        self.counts = counts
        self.values = values
        self.reward_squared = reward_squared
        return
  
    def initialize(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        self.reward_squared = [0.0 for col in range(n_arms)]
        return
  

    def select_arm(self):
        n_arms = len(self.counts)
        for arm in range(n_arms):
            if self.counts[arm] == 0: 
                return arm

        ucb_values = [0.0 for arm in range(n_arms)]
        total_counts = sum(self.counts)
        
        for arm in range(n_arms):
            variance_bound = self.reward_squared[arm] / float(self.counts[arm]) - float(self.values[arm]) ** 2
            variance_bound += math.sqrt((2 *math.log(total_counts)) / float(self.counts[arm]))        
            bonus = math.sqrt(np.min([variance_bound, 1 / 4]) *(math.log(total_counts)) / float(self.counts[arm]))
            ucb_values[arm] = self.values[arm] + bonus
        return ind_max(ucb_values)
  
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]
        self.reward_squared[chosen_arm] += reward**2 
        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value
        return

## Bayes UCB

In [ ]:
from scipy.stats import beta
class BayesUCB():
    def __init__(self, S, F):
        self.S = S # number successes
        self.F = F # number failures 
        return
  
    def initialize(self, n_arms):
        self.S = [1 for col in range(n_arms)] 
        self.F = [1 for col in range(n_arms)] 
        return

    def select_arm(self):
        n_arms = len(self.S)
        bound = np.zeros(n_arms) 
        for i in range(n_arms):
            bound[i] = self.S[i] / (self.S[i] + self.F[i]) + (beta.std(self.S[i],self.F[i]))
        return ind_max(bound)
  
    def update(self, chosen_arm, reward):
        if reward == 1:
            self.S[chosen_arm] += 1
        else:
            self.F[chosen_arm] += 1
        return

## Thompson Sampling

In [ ]:
class Thompson():
    def __init__(self, S, F): 
        self.S = S 
        self.F = F
        return
  
    def initialize(self, n_arms):
        self.S = [1 for col in range(n_arms)]  
        self.F = [1 for col in range(n_arms)] 
        return

    def select_arm(self):
        n_arms = len(self.S)
        probs = np.zeros(n_arms)  # sampling prob win, each machine
        for i in range(n_arms):
            probs[i] = np.random.beta(self.S[i], self.F[i], size=1)[0] 
        return ind_max(probs)
  
    def update(self, chosen_arm, reward):
        if reward == 1:
            self.S[chosen_arm] += 1
        else:
            self.F[chosen_arm] += 1
        return